In [1]:
#ONION NETWORK WITH SCALE-FREE DEGREE DISTRIBUTION

#create probability distribution P(k)=a*k**(-gamma)
#draw N numbers from this distribution: assign degree to each vertex (N vertices in network)
#assign layer indices s_i according to degree (smallest degree=0, etc.)
#join stubs with probability: p_ij = 1/(1+a*((s_i)-(s_j)))
#reshuffling procedure for unused stubs: break up existing connections and make swaps

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import scipy.special
from scipy.special import zeta, polygamma, factorial

In [3]:
#GENERATE PROBABILITY DISTRIBUTION FOR DEGREES OF SCALE-FREE NETWORK
def degree_distribution(cutoff,gamma,xmin,N):
    probabilities=np.zeros(cutoff+1)
    normalization=scipy.special.zeta(gamma)

    for i in range(cutoff):
        probabilities[i]=(i+1)**(-gamma)
        probabilities[i]/=normalization

    psum=sum(probabilities)
    remainder=1-sum(probabilities)

    probabilities[cutoff]=remainder

    degrees=np.linspace(1,cutoff+1,cutoff+1)
    degree_distribution=np.random.choice(degrees,N,p=probabilities).astype(int)

    #print(degree_distribution)
    return degree_distribution

In [4]:
#CREATE NETWORK WITH N NODES AND NO EDGES
def onionnetwork(N):
    G=nx.Graph()
    for i in range(N):
        G.add_node(i)
    return G

In [38]:
def node_layer_indices(G, degree_sequence):
    attr=dict()
    degrees=[]
    for x in degree_sequence:
        if x not in degrees:
            degrees.append(x)
    degrees.sort()
    
    for i in range(len(degrees)):
        mindegree=degrees[i]
        for node in G.nodes():
            if degree_sequence[node]==mindegree:
                nodeDict=dict()
                nodeDict["layer"]=i
                attr[node]=nodeDict
    nx.set_node_attributes(G,attr)
    return(G)            

In [22]:
def node_layer_indices_rand(G,distribution):
    attr=dict()
    degrees=[]
    for x in distribution:
        if x not in degrees:
            degrees.append(x)
    degrees.sort()
    
    for i in range(len(degrees)):
        mindegree=degrees[i]
        for node in G.nodes():
            if G.degree(node)==mindegree:
                nodeDict=dict()
                nodeDict["layer"]=i
                attr[node]=nodeDict
    nx.set_node_attributes(G,attr)
    return(G)   

In [1]:
#JOIN "STUBS" OF NODE LINKS
def join_stubs(G,distribution,error):
    stubcount=np.copy(distribution)
    for i in G.nodes():
        for j in G.nodes():
            if i!=j:
                if stubcount[i]>0 and stubcount[j]>0:
                    #calculate probability of joining: 
                    diff=np.absolute(G.nodes[i]["layer"]-G.nodes[j]["layer"])
                    p=1/(1+diff)
                    x=np.random.uniform()                        
                    #if random number smaller than p, join nodes, decrease stubcounts
                    if x<=p:
                        G.add_edge(i,j)
                        stubcount[i]=stubcount[i]-1
                        stubcount[j]=stubcount[j]-1
    
    while sum(stubcount)>error:
        stubs=[]
        edgelist=[]
        for i in G.nodes():
            if stubcount[i]>0:
                stubs.append(i)
        for x in G.edges():
            edgelist.append(x)
        swap=np.random.choice(stubs,2)
        max=len(edgelist)
        index=np.random.randint(0,max)
        swapedge=edgelist[index]
        node1=swap[0]
        node2=swap[1]
        node3=swapedge[0]
        node4=swapedge[1]
        G.remove_edge(node3,node4)
        #print("removed:",(node1,node2))
        G.add_edge(node3,node1)
        G.add_edge(node4,node2)
        stubcount[node1]=stubcount[node1]-1
        stubcount[node2]=stubcount[node2]-1
        #print(sum(stubcount))
                    
    return G

In [8]:
#robustness of network
def get_robustness(network):
    summa=0
    networksize=network.number_of_nodes()
    for i in range(Gc.number_of_nodes()):
        graph=Gc.copy()
        list=np.array(graph.nodes())
        nodelist=np.random.choice(list,i, replace=False)
        for j in nodelist:
            graph.remove_node(j)
        connected=max(nx.connected_component_subgraphs(graph), key=len)
        size=len(connected)
        summa+=size/networksize
    #print(summa/size)
    return(summa/networksize)

In [9]:
#COUNT NUMBER OF SHELLS IN NETWORK
def countshells(G):
    G.remove_edges_from(nx.selfloop_edges(G))
    list=[0]
    i=1
    shells=0
    while list!=[]:
        list=[]
        for node in nx.k_shell(G,i):
            list.append(node)
        shells+=1
        i+=1
    return shells

In [10]:
#CREATE DICTIONARY OF NODES:K-INDICES, SET AS NODE ATTRIBUTES IN GRAPH
def k_core_decomposition(G, shell_count):
    attr=dict()
    
    for i in range(shell_count):
        for node in nx.k_shell(G,i).nodes():
            nodeDict=dict()
            nodeDict["k-shell index"]=i
            attr[node]=nodeDict
           
    nx.set_node_attributes(G,attr)
    
    return G

In [11]:
def k_color_map(G):
    color=[]
    labels=dict()
    for node in G.nodes():
        color.append(G.nodes[node]["k-shell index"])
        labels[node]=str(G.nodes[node]["k-shell index"])
    netLayout=nx.spring_layout(G,weight='weight')
    plt.figure(figsize=(5,5))
    nx.draw(G,netLayout,node_color=color,cmap="OrRd",node_size=50)
    #nx.draw_networkx_labels(G,netLayout,labels,font_size=16)#a második egy ilyen positionös cucc

In [12]:
#GET DICTIONARY OF NODES IN SHELLS
def shells(G, shell_count):
    shells=dict()
    for i in range(shell_count):
        list=[]
        for node in nx.k_shell(G,i).nodes():
            list.append(node)
        shells[i]=list
    return shells

In [13]:
def layer_dict(G): #returns dictionary with layer indices as keys, and list of nodes as values
    attributes=nx.get_node_attributes(G,"layer")
    layerdict=dict()
    for x in range(max(attributes.values())+1):
        layerdict[x]=[]
    for k,v in attributes.items():
        layerdict[v].append(k)
    
    return(layerdict)

In [14]:
def highest_layer(G): #index of highest layer in network = number of layers
    highest_layer=max(nx.get_node_attributes(G,"layer").values())
    return highest_layer

In [31]:
def onion_similarity(G, layerdict,highestlayer):
    totalerror=[]
    for i in range(highestlayer): #highestlayer=number of layers
        for j in range(highestlayer):
            if i>=j: #only do each combination once :P
                links=0
                if layerdict[i]!=[] and layerdict[j]!=[]: #if there are nodes in the layer
                    for p in layerdict[i]:
                        for q in layerdict[j]:
                            if p>=q and (p,q) in G.edges():
                                links+=1 #check if p and q are linked. if yes, add 1 to number of links.
                    #print("links between",i,"and",j,":",links)
                    pi=len(layerdict[i]) #number of nodes in layer i 
                    pj=len(layerdict[j]) #number of nodes in layer j
                    possible=((pi)*(pj))/2 #number of links possible between the two layers
                    real_prob=links/possible #probability of any two nodes, one in i, one in j, being linked.
                    exp_prob=1/(1+np.absolute(i-j)) #probability of there being a link if the network were onion-type.
                    error=np.absolute(exp_prob-real_prob) #difference between the actual and the "onion-type" probability
                    totalerror.append(error) #add this difference to the list totalerror
    errorsum=0
    mean=sum(totalerror)/len(totalerror)
    for x in totalerror: 
        squared_diff=(x-mean)**2
        errorsum+=squared_diff
    mean_squared_error=(errorsum/len(totalerror))**0.5
    return mean_squared_error/(highestlayer)

In [33]:
def randomerror(x,degree_distribution):
    
    degrees=degree_distribution
    if sum(degrees)%2==1:
        degrees[0]+=1
    G=nx.configuration_model(degrees)
    node_layer_indices_rand(G,degrees)

    layerdict=layer_dict(G)
    highestlayer=highest_layer(G)
    print("random highest layer",highestlayer)
    onionerror_random=onion_similarity(G,layerdict,highestlayer)
    
    return onionerror_random

In [36]:
def onionerror(x,degree_distribution):
    
    g=onionnetwork(x)
    node_layer_indices(g,degree_distribution)

    H=join_stubs(g,degrees,125)
    H=max(nx.connected_component_subgraphs(H), key=len)
    
    layerdict=layer_dict(H)
    highestlayer=highest_layer(H)
    print("onion highest layer",highestlayer)
    onionerror_onion=onion_similarity(H,layerdict,highestlayer)
    
    return onionerror_onion

In [18]:
def expected_links(G,a):
    layers=layer_dict(G)
    highestlayer=max(nx.get_node_attributes(G,"layer").values())
    population=[0]*highestlayer
    for x in range(highestlayer):
        population[x]=len(layers[x])
    sum=0
    for i in range(highestlayer):
        for j in range(highestlayer):
            if i>j:
                diff=np.absolute(i-j)
                p=1/(1+(a*diff))
                #print(p)
                ni=len(layers[i])
                nj=len(layers[j])
                sum+=(ni*nj)*p
    return sum

In [39]:
for N in [50,100,200,500]:
    print("number of nodes:",N)
    for p in [0.01,0.05,0.1,0.15,0.2,0.5,0.7]:
        print("p:",p)
        degrees=degree_distribution(100,2,2,N)
        err_onion=onionerror(N,degree_distribution)
        #print("onion:", err_onion)
        err_rand=randomerror(N,degree_distribution)
        #print("random:",err_rand)
        print(err_onion/err_rand)

number of nodes: 50
p: 0.01


TypeError: 'function' object is not iterable

In [ ]:
#degrees=degree_distribution(100,2,2,500)
#network=onionnetwork(500)
#node_layer_indices(network,degrees)
#join_stubs(network, degrees,125)
#nx.get_node_attributes(network,"layer")
#plt.figure(figsize=(10,10))
#Gc = max(nx.connected_component_subgraphs(network), key=len)
#plt.figure(figsize=(8,8))
#nx.draw(Gc, node_size=50)

In [ ]:
#simulations=20
#for x in range(simulations):
 #   degrees=degree_distribution(100,2,2,500)
  #  network=onionnetwork(500)
   # node_layer_indices(network,degrees)
    #join_stubs(network,degrees,125)
    #plt.figure(figsize=(10,10))
    #Gc = max(nx.connected_component_subgraphs(network), key=len)
    #nx.draw(Gc, node_size=50)
    #print("robustness:",get_robustness(Gc))